In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# mean encoding
# this first dummifies the target then creates mean encodings.
def mean_encoding(columns,df,target,df_test): # here columns is the name of the columns to encode and df is the dataframe target is the column name for the target variable
    mean_en = pd.DataFrame()
    mean_en_test = pd.DataFrame()
    temp = pd.DataFrame(df[target])
    n = temp[target].nunique()
    un = temp[target].unique()
    count = 0
    new_target = pd.DataFrame()
    for i in range(n):
        tempx = []
        for j in range(df.shape[0]):
            if un[i] == temp.iloc[j,0]:
                tempx.append(1)
            else:
                tempx.append(0)
        new_target[un[i]] = tempx
    df = df.drop(target,axis = 1)
    temp2 = pd.concat([df,new_target],axis = 1)
    for i in columns:
        for j in range(n):
            temp = temp2.groupby(i)[un[j]].mean()
            mean_en[f"{i}+{un[j]}_mean_encoding"] = temp2[i].map(temp)
            mean_en_test[f"{i}+{un[j]}_mean_encoding"] = df_test[i].map(temp)
            if mean_en[f"{i}+{un[j]}_mean_encoding"].isnull().sum()>0:
                glob_mean = temp2[un[j]].mean()
                mean_en[f"{i}+{un[j]}_mean_encoding"] = mean_en[f"{i}+{un[j]}_mean_encoding"].fillna(glob_mean)
                mean_en_test[f"{i}+{un[j]}_mean_encoding"] = mean_en_test[f"{i}+{un[j]}_mean_encoding"].fillna(glob_mean)
    return mean_en,mean_en_test

In [ ]:
# AUTOMATED TRAINING FUNCTIONS
def train_lgbm_mean(df_train,df_test,lgbm_para,cols,ytrain,target_col_name):
    """
    df_train:dataframe to train
    df_test:test dataframe
    lgbm_para:all the parameters required for the lightgbm
    cols:the columns for which you would like to do mean encoding
    ytrain: target for training data
    target_col_name:target column name
    
    
    To run this convert and convert2 functions must be there
    
    """
    
    gbm = lightgbm
    
    columns_encode = ["Violations","Accident_Type_Code","Days_Since_Inspection","Total_Safety_Complaints"]

    df_cols = list(df_train.columns)   

    temp_df = pd.concat([df_train,ytrain],axis = 1)
    print(temp_df.shape)
    df_cols.append(target_col_name)

    temp_df.columns = df_cols

    x,z = mean_encoding(cols,temp_df,target_col_name,df_test)

    y_c = convert2(y)
    
    train = pd.concat([temp_df,x],axis = 1)
    test = pd.concat([df_test,z],axis = 1)
    train = train.drop(target_col_name,axis =1)
    train_d = gbm.Dataset(train.values,label = y_c)
    lgbm = gbm.train(lgbm_para,train_set=train_d)
    
    return lgbm

In [ ]:
def train_lgbm(df_train,lgbm_para,ytrain):
    """
    df_train:dataframe to train
    df_test:test dataframe
    lgbm_para:all the parameters required for the lightgbm
    cols:the columns for which you would like to do mean encoding
    ytrain: target for training data
    target_col_name:target column name
    
    
    To run this convert and convert2 functions must be there
    
    """
    
    gbm = lightgbm
    y_c = convert2(y)
    train_d = gbm.Dataset(df_train.values,label = y_c)
    lgbm = gbm.train(lgbm_para,train_set=train_d)
    
    return lgbm
    

In [ ]:
def train_nn(params,df,y_nn):
    
    reg = l2(params["reg"])
    model = Sequential()
    model.add(Dense(141,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg))
    model.add(BatchNormalization())
    model.add(Dense(1000,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg))
    model.add(BatchNormalization())
    model.add(Dense(1000,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg))
    model.add(BatchNormalization())
    model.add(Dense(1000,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer =reg))
    model.add(BatchNormalization())
    model.add(Dense(1000,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg))
    model.add(BatchNormalization())
    model.add(Dense(500,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer = reg))
    model.add(BatchNormalization())
    model.add(Dense(4,activation=tf.nn.softmax))
    
    adam = keras.optimizers.Adam(learning_rate=params["learning_rate"], beta_1=0.9, beta_2=0.999, amsgrad=False)

    mc = ModelCheckpoint(r'E:\Data Science\Kaggle Competitions dataset\Hackerearth Airplane Accident/best_model.h5', monitor='categorical_crossentropy', mode='min', verbose=1, save_best_only=True)        
    reduce_lr = ReduceLROnPlateau(monitor='categorical_crossentropy',mode = "min", factor=0.params["factor"],patience=params["patience"], min_lr=0.000001)
    
    model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['categorical_crossentropy'])

    history = model.fit(df,y_nn.values, epochs=params["epochs"],batch_size = params["batch_size"],callbacks = [mc,reduce_lr],verbose =1)

    
    return model

In [ ]:
def train_rf(params,df,y):
    
    rf = RandomForestClassifier(n_jobs = -1,n_estimators = params["n_estimators"],min_samples_split = params["min_samples_split"],max_depth = params["max_depth"],max_features = params["max_features"])
    rf.fit(df,y)
    
    return rf
    

In [ ]:
def train_LR(params,df,y):
    
    log_reg = LogisticRegression(solver = "sag",max_iter=params["max_iter"],C=params["C"],penalty="l2",multi_class="ovr",n_jobs=8)
    log_reg.fit(df,y)
    return log_reg

In [ ]:
def train_KNN(params,df,y):
    knn = KNeighborsClassifier(n_neighbors = params["n_neighbors"],n_jobs = -1)
    knn.fit(df,y)
    
    return knn

In [ ]:
def train_gnb(params,df,y):
    gnb = GaussianNB()
    gnb.fit(df,y)
    
    return gnb

In [ ]:
# FUNCtion to extract feature importances

imp_feat = lgbm_test.feature_importance(importance_type = "gain") # these are the feature importances
feat_name = lgbm_test.feature_name() # these are the feature names

# lets make a dictionary

feat_dict = {}
for c,i in enumerate(imp_feat):
    feat_dict[i] = feat_name[c]
z = sorted(feat_dict.keys(),reverse = True)
count = 0
important_features = [] #this contains all the important features names in a list
for i in z:
    if count<50: # number of features to extract
        important_features.append(feat_dict[i])
    else:
        break
    count =count+1


In [ ]:
# KERAS TRANSFERRING WEIGHTS OF 1 LAYER TO ANOTHER
# here we get weights from saved_auto and map it to temp
for c,i in enumerate(temp.layers):
    temp.layers[c].set_weights(saved_auto.layers[c].get_weights())
    

In [ ]:
# TO INSTALL EFFICIENT NET
pip install keras_efficientnets #(EASIEST WAY)
# also look at efficient net from pip can be a seperate module idk why
from keras_efficientnets import EfficientNetB5
effnet = EfficientNetB5(input_shape=(456,456,3),
                        weights='imagenet',
                        include_top=False)

In [ ]:
#For installing efficientnet in tensorflow 2.0

pip install -U efficientnet

import efficientnet.tfkeras as efn 
effnet = efn.EfficientNetB3(input_shape = (300,300,3),weights='imagenet',include_top = False)

In [ ]:
# CV K fold
cv = KFold(4, random_state=42,shuffle = True)

for train, test in cv.split(normal):
    X_train, X_test = normal[train], normal[test]
    y_train, y_test = y_nn.values[train], y_nn.values[test]

In [ ]:
# Stratified K fold
cv = StratifiedKFold(4, random_state=42,shuffle = True)

for train, test in cv.split(normal,temp_y):
    X_train, X_test = normal[train], normal[test]
    y_train, y_test = y_nn.values[train], y_nn.values[test]

In [ ]:
# Removing Outliers

# METHOD 1
import seaborn as sns
sns.boxplot(x=boston_df['DIS']) # This is a boxplot Anything that appears to be a point in the plot is considered as an outlier

# METHOD 2
# Removing outliers

from scipy import stats
import numpy as np
z= []
for i in feat_df.columns:
    temp = np.abs(stats.zscore(feat_df[i])) #This gives the z scores of the particular df
    z.append(temp)

# Z Scores that are above or below 3,-3 are considered to be outliers
# Z score kind of measures how many standard deviations is a point far of
to_del = {0} # rows to be deleted

# The code below deletes the outliers
# deletion is not always recommended capping can also be used instead of deletion
for i in (z):
    for c,j in enumerate(i):
        if j >3:
            to_del.add(c) 


In [ ]:
# PERFECT FUNCTION FOR MEAN target ENCODING

# this first dummifies the target then creates mean encodings for both train and test
def mean_encoding(columns,df,target,df_test): # here columns is the name of the columns to encode and df is the dataframe target is the column name for the target variable
    mean_en = pd.DataFrame()
    mean_en_test = pd.DataFrame()
    temp = pd.DataFrame(df[target])
    n = temp[target].nunique()
    un = temp[target].unique()
    count = 0
    new_target = pd.DataFrame()
    for i in range(n):
        tempx = []
        for j in range(df.shape[0]):
            if un[i] == temp.iloc[j,0]:
                tempx.append(1)
            else:
                tempx.append(0)
        new_target[un[i]] = tempx
    df = df.drop(target,axis = 1)
    temp2 = pd.concat([df,new_target],axis = 1)
    for i in columns:
        for j in range(n):
            temp = temp2.groupby(i)[un[j]].mean()
            mean_en[f"{i}+{un[j]}_mean_encoding"] = temp2[i].map(temp)
            mean_en_test[f"{i}+{un[j]}_mean_encoding"] = df_test[i].map(temp)
            if mean_en[f"{i}+{un[j]}_mean_encoding"].isnull().sum()>0:
                glob_mean = temp2[un[j]].mean()
                mean_en[f"{i}+{un[j]}_mean_encoding"] = mean_en[f"{i}+{un[j]}_mean_encoding"].fillna(glob_mean)
                mean_en_test[f"{i}+{un[j]}_mean_encoding"] = mean_en_test[f"{i}+{un[j]}_mean_encoding"].fillna(glob_mean)
    return mean_en,mean_en_test

In [ ]:
# CROSS VALIDATION WITH MEAN ENCODING

cv = StratifiedKFold(4, random_state=42,shuffle = True)
li =[]
for train, test in cv.split(df_feat,y):
    X_train, X_test = df_feat.values[train], df_feat.values[test]
    y_train, y_test = y[train], y[test]
    y_c = convert2(y_train)
    y_c_t = convert2(y_test)
    
    df_cols = list(df_feat.columns)
    
    temp_train = pd.DataFrame(X_train)
    temp_train.columns = df_cols
    temp_y = pd.DataFrame(y_train)
    temp_y.columns = ["Severity"]
    temp_test = pd.DataFrame(X_test)
    temp_test.columns = df_cols
    
    columns_encode = ["Violations","Accident_Type_Code","Days_Since_Inspection","Total_Safety_Complaints"]
    
    temp_df = pd.DataFrame(np.hstack([temp_train,temp_y]))
    df_cols.append("Severity")
    temp_df.columns = df_cols
    x,z = mean_encoding(columns_encode,temp_df,"Severity",temp_test)
    
    train = pd.concat([temp_train,x],axis = 1)
    test = pd.concat([temp_test,z],axis = 1)
    #train = train.drop("Severity",axis =1)
    train_d = gbm.Dataset(train.values,label = y_c)
    test_d= gbm.Dataset(test.values,label = y_c_t)
    lgbm = gbm.train(all_params,train_set=train_d,valid_sets=test_d)
    temp = lgbm.predict(test.values)
    li.append(f1_score(y_c_t,convert2(convert(temp)),average = "weighted"))
    temp1 = f1_score(y_c_t,convert2(convert(temp)),average = "weighted")
output = (sum(li)/4)
print(output)

In [ ]:
#FEATURE ENGINEERING # GENERATING A LOT OF RANDOM FEATURES
groupby_feat = pd.DataFrame()
cat_col = ["name_of_drug","use_case_for_drug"] #categorical columns for which we want to create features
num_col = ["number_of_times_prescribed","effectiveness_rating"] #Numerical columns for which we want to create features for the categorical columns
aggs = [np.mean,max,min,np.std,sum] #functions we want to apply for the columns
fun_name_dict = {
    np.mean:'mean',
    max:'max',
    min:'min',
    np.std:'std',
    sum:'sum'
}

for cat in cat_col:
    for num in num_col:
        for fun in aggs:
            fun1 = fun_name_dict[fun]
            groupby_feat[f'feature_{cat}_{num}_{fun1}'] = df.groupby(cat)[num].transform(fun)

            # FOR BETTER SEE GENERATING RANDOM FEATURES IN KAGGLE PREDICTING MOLECULAR PROPERTIES CHALLENGE

In [ ]:
# random feature functions in Cross Validation
# used for creating features and mapping them to test data in the cv
def create_features(df):
    groupby_feat = pd.DataFrame()
    cat_col = ["name_of_drug","use_case_for_drug"] #categorical columns for which we want to create features
    num_col = ["number_of_times_prescribed","effectiveness_rating"] #Numerical columns for which we want to create features for the categorical columns
    aggs = [np.mean,max,min,np.std,sum] #functions we want to apply for the columns
    fun_name_dict = {
        np.mean:'mean',
        max:'max',
        min:'min',
        np.std:'std',
        sum:'sum'
    }
    feat_dict = {}
    
    for cat in cat_col:
        for num in num_col:
            for fun in aggs:
                fun1 = fun_name_dict[fun]
                groupby_feat[f'feature_{cat}_{num}_{fun1}'] = df.groupby(cat)[num].transform(fun)
                feat_dict[f'feature_{cat}_{num}_{fun1}'] = df.groupby(cat)[num].apply(fun)
    diff = (pd.to_datetime('01-Jan-20', format='%d-%b-%y') - pd.to_datetime(df['drug_approved_by_UIC'], format='%d-%b-%y')).dt.days
   
    groupby_feat['time_reference'] = diff
    all_feat = pd.concat([df,groupby_feat],axis = 1)
    
    return all_feat,feat_dict,cat_col

import re
def map_features(data,feat_dictionary,cat): # used for mapping features to test data
    for i in feat_dictionary.keys():
        for j in cat:
            if re.search(j,i):
                temp_dict_1 = dict(zip(feat_dictionary[i].index, feat_dictionary[i].values))
                data[i] = data[j].map(temp_dict_1)
                print(j,i)
    return data

In [ ]:
# interesting features
# line features Safety score and days since inspection
# making graph
# variables to output
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
r = 5 # rows
c=5 #columns
t = 0 #temporary counter
f = plt.figure(figsize=(60,60))
gs = f.add_gridspec(r,c)
count = 0
colors = ['red','green','blue','yellow']

for i in range(len(df.columns)):
    for j in range(i,len(df.columns)):
        if count%r==0:
            t=t+1
        f.add_subplot(gs[count%r,t])
        plt.scatter(df[df.columns[i]],df[df.columns[j]],c=y_conv, cmap=ListedColormap(colors))
        plt.title(f'X = {df.columns[i]}+ Y ={df.columns[j]}')
        count = count+1

In [ ]:
#HYPEROPT
from hyperopt import hp, tpe, fmin

fixed_param = {
               "objective":"multiclass",
               "metric":"multi_logloss",
               "num_class":4,
               "boosting_type":"gbdt"
              }

# This is the space that you want this function to explore
space_gbm = {
             'max_depth': hp.uniform('max_depth',5,40),
             'num_iterations': hp.uniform('num_iterations',50, 600),
             'num_leaves': hp.uniform('num_leaves',10, 120),
             'min_data_in_leaf':hp.uniform('min_data_in_leaf',50, 500),
             'learning_rate': hp.uniform('learning_rate',0.01, 0.1)
            }
# THIS is the objective function that we want to minimize (note that we are multiplying the output by -1 as this function is always minimized never maximized)
def objective_gbm(params):
    cv = StratifiedKFold(4, random_state=42,shuffle = True)
    li =[]
    # THis is CV in hyperopt
    for train, test in cv.split(df,y):
        X_train, X_test = df.values[train], df.values[test]
        y_train, y_test = y.values[train], y.values[test]
        y_c = convert2(y_train)
        y_c_t = convert2(y_test)
    
        params = {'num_iterations': int(params['num_iterations']),'max_depth': int(params['max_depth']),'num_leaves': int(params['num_leaves']),'min_data_in_leaf': int(params['min_data_in_leaf']),'learning_rate': float(params['learning_rate'])}
        all_params = fixed_param
        for i in params:
            all_params[i] = params[i]
    
        train_d = gbm.Dataset(X_train,label = y_c)
        test_d= gbm.Dataset(X_test,label = y_c_t)
        lgbm = gbm.train(all_params,train_set=train_d,valid_sets=test_d)
        temp = lgbm.predict(X_test)
        li.append(f1_score(y_c_t,convert2(convert(temp)),average = "weighted"))
        temp1 = f1_score(y_c_t,convert2(convert(temp)),average = "weighted")
    output = (sum(li)/4)
    print(output)
    return -1*output
# THis is the actual function that is run by this function
best_gbm = fmin(objective_gbm,
    space=space_gbm,
    algo=tpe.suggest,
    max_evals=50)


In [ ]:
# ORIGINAL FEATURE ENGINEERING FUNCTION
def create_features_full(df):
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')
    df['molecule_dist_mean'] = df.groupby('molecule_name')['dist'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['dist'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['dist'].transform('max')
    df['molecule_dist_std'] = df.groupby('molecule_name')['dist'].transform('std')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')

    num_cols = ['x_1', 'y_1', 'z_1', 'dist', 'dist_x', 'dist_y', 'dist_z'] # numeric columns
    cat_cols = ['atom_index_0', 'atom_index_1', 'type', 'atom_1', 'type_0'] # categorical columns
    aggs = ['mean', 'max', 'std', 'min']
    for col in cat_cols:
        df[f'molecule_{col}_count'] = df.groupby('molecule_name')[col].transform('count')

    for cat_col in tqdm_notebook(cat_cols):
        for num_col in num_cols:
            for agg in aggs:
                df[f'molecule_{cat_col}_{num_col}_{agg}'] = df.groupby(['molecule_name', cat_col])[num_col].transform(agg)
                df[f'molecule_{cat_col}_{num_col}_{agg}_diff'] = df[f'molecule_{cat_col}_{num_col}_{agg}'] - df[num_col]
                df[f'molecule_{cat_col}_{num_col}_{agg}_div'] = df[f'molecule_{cat_col}_{num_col}_{agg}'] / df[num_col]

    df = reduce_mem_usage(df)
    return df

In [ ]:
#csv download link
from IPython.display import HTML

def create_download_link(title = "Download CSV file", filename = "linear_prediction.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='linear_prediction.csv')

In [ ]:
# preprocessing DATA FOR MODE (FILLING NAN's BY THE MODE)
df[i] = df[i].fillna(df[i].mode()[0])

In [ ]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
LDA = LinearDiscriminantAnalysis(n_components = 50)
LDA.fit(normal_train,y_normal_train)
LDA_data = LDA.transform(normal)

In [ ]:
#SAVING A NN MODEL THROUGH KERAS
model_json = model4.to_json()
with open("overfitNN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model4.save_weights("overfitNN.h5")
print("Saved model to disk")

In [ ]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(normal, y_normal, test_size=0.2, random_state=42)

In [ ]:
#model storing through pickle
filename = "Linear_regression.sav"
pickle.dump(Lreg, open(filename, 'wb'))


In [ ]:
#SAVED MODEL DOWNLOAD
def create_download_link(title = "Download Model", filename = "Linear_regression.sav"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='Linear_regression.sav')

In [ ]:
#PCA


In [ ]:
#Basic Neural Network
from keras.regularizers import l2 as reg2,l1 as reg1
from keras.models import Sequential
from keras.layers import Dense,Activation,BatchNormalization,Dropout


model4 = Sequential()
model4.add(Dense(200,activation=tf.nn.relu ,kernel_initializer='lecun_uniform'))
model4.add(BatchNormalization())
model4.add(Dense(10000,activation=tf.nn.tanh ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(Dropout(rate = 0.2))
model4.add(BatchNormalization())
model4.add(Dense(2000,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(Dropout(rate = 0.2))
model4.add(BatchNormalization())
model4.add(Dense(1000,activation=tf.nn.tanh ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(BatchNormalization())
model4.add(Dense(500,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(BatchNormalization())
model4.add(Dense(100,activation=tf.nn.tanh ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(BatchNormalization())
model4.add(Dense(100,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(BatchNormalization())
model4.add(Dense(100,activation=tf.nn.relu ,kernel_initializer='lecun_uniform',kernel_regularizer=reg2))
model4.add(BatchNormalization())
model4.add(Dense(1,activation=tf.nn.sigmoid ,kernel_initializer='lecun_uniform'))



In [ ]:
model4.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history4 = model4.fit(X_train, y_train.values, epochs=30,batch_size = 1024,validation_split = 0.1)

In [ ]:
#USEFUL TRANSFER LEARNING FUNCTIONS
#picking specific layers and weights of a network
pre_trained = []
for i in automodel.layers[:8]:
    pre_trained.append(i)

# adding layers of pre_trained to new model
new_model = Sequential()
for i in pre_trained:
    new_model.add(i)

In [ ]:
#freezing layers
for layer in new_model.layers:
    layer.trainable=False

In [ ]:
#plotting loss

plt.plot(history4.history['loss'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
#plotting ROC
#ROC defines how good the model is at separating classes
from sklearn.metrics import roc_curve, auc
fpr, tpr, thr = roc_curve(y_train, model4.predict_proba(X_train))
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic Plot for training data')
auc(fpr, tpr)

In [ ]:
#SOME CNN FUNCTIONS


#this function is useful for getting predictions after creating a model with a specific generator
def get_preds_and_labels(model, generator):
    """
    Get predictions and labels from the generator
    """
    preds = []
    labels = []
    for _ in range(int(np.ceil(generator.samples / BATCH_SIZE))):
        x, y = next(generator)
        preds.append(model.predict(x))
        labels.append(y)
    # Flatten list of numpy arrays
    return np.concatenate(preds).ravel(), np.concatenate(labels).ravel()



# This is the metrics class and it has functions for keras callback 
#on train begin happens at the beginning of training 
# on epoch end happens at the end of each epoch
#in this on epoch end function we save the model if the validation score improves
# for the specified metric (in this case kappa)
class Metrics(Callback):
    """
    A custom Keras callback for saving the best model
    according to the Quadratic Weighted Kappa (QWK) metric
    """
    def on_train_begin(self, logs={}):
        """
        Initialize list of QWK scores on validation data
        """
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        """
        Gets QWK score on the validation data
        """
        # Get predictions and convert to integers
        y_pred, labels = get_preds_and_labels(model, val_generator)
        y_pred = np.rint(y_pred).astype(np.uint8).clip(0, 4)
        # We can use sklearns implementation of QWK straight out of the box
        # as long as we specify weights as 'quadratic'
        _val_kappa = cohen_kappa_score(labels, y_pred, weights='quadratic')
        self.val_kappas.append(_val_kappa)
        print(f"val_kappa: {round(_val_kappa, 4)}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model.")
            self.model.save(SAVED_MODEL_NAME)
        return

In [ ]:
# some pre processing of the images

# to resize the image for a specific size
image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
# To add gaussian noise (it somehow makes the model rhobust)
image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    

In [ ]:
# ideally we should make a function for pre processing which would pre process
# the images with diff pre processings. This should be done to be able to use
# the pre processing function in the generator

def preprocess_image(image):
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

In [ ]:
BATCH_SIZE = 4
#These are very important functions for data augmentation and generation while using
# in the training and testing
# Add Image augmentation to our generator
# see ImageDataGenerator keras documentation
train_datagen = ImageDataGenerator(rotation_range=360,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.15,
                                   preprocessing_function=preprocess_image, #this is the pre-processing function that will be applied to the image
                                   rescale=1 / 128.)

# Use the dataframe to define train and validation generators
train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='id_code', 
                                                    y_col='diagnosis',
                                                    directory = TRAIN_IMG_PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='other', 
                                                    subset='training')

val_generator = train_datagen.flow_from_dataframe(train_df, 
                                                  x_col='id_code', 
                                                  y_col='diagnosis',
                                                  directory = TRAIN_IMG_PATH,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='other',
                                                  subset='validation')

In [ ]:
# FINAL MODEL HERE with early stopping and some other stuff that is put into
# callbacks.

# For tracking Quadratic Weighted Kappa score
kappa_metrics = Metrics()
# Monitor MSE to avoid overfitting and save best model
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=12)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=4, 
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.0001)

# Begin training
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=35,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // BATCH_SIZE,
                    callbacks=[kappa_metrics, es, rlr])

In [2]:
# for using EMBEDDINGS learned on some corpus #TRANSFER LEARNING
# use this function for mapping the words to the embeddings.
# Taken from https://www.kaggle.com/bertcarremans/using-word-embeddings-for-sentiment-analysis
# just see this link it is flawless 

#Using pre-trained word embeddings
#Because the training data is not so big, 
#the model might not be able to learn good embeddings for the sentiment analysis. Luckily we can load pre-trained word embeddings built on a much larger training data.
#The GloVe database contains multiple pre-trained word embeddings, and more specific embeddings trained on tweets.#


glove_file = 'glove.twitter.27B.' + str(GLOVE_DIM) + 'd.txt' #the embedding file name
glove_dir = 'glove-global-vectors-for-word-representation/' # the directory where the embedding is stored
emb_dict = {}
glove = open(input_path / glove_dir / glove_file)
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

#To feed this into an Embedding layer, we need to build a matrix containing the
#words in the tweets and their representative word embedding. 
#So this matrix will be of shape (NB_WORDS, GLOVE_DIM)

#Before using embeddings we must first make a map for tokenization of the data
#This means that each word will be replaced by a token number which will uniquely
#represent that word with a unique token
tk = Tokenizer(num_words=NB_WORDS,
               filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
               lower=True,
               split=" ")
tk.fit_on_texts(X_train)

X_train_seq = tk.texts_to_sequences(X_train)
X_test_seq = tk.texts_to_sequences(X_test)


#ONLY USE THE BELOW FUNCTION AFTER TOKENIZATION

emb_matrix = np.zeros((NB_WORDS, GLOVE_DIM))

for w, i in tk.word_index.items():
    # The word_index contains a token for all words of the training data so we need to limit that
    if i < NB_WORDS:
        vect = emb_dict.get(w)
        # Check if the word from the training data occurs in the GloVe word embeddings
        # Otherwise the vector is kept with only zeros
        if vect is not None:
            emb_matrix[i] = vect
    else:
        break

# Below is a sample model We can replace with our own model.

glove_model = models.Sequential()
glove_model.add(layers.Embedding(NB_WORDS, GLOVE_DIM, input_length=MAX_LEN)) #nb words is the number of words (size of dictionary) MAX LEN is the max length of a sequence
glove_model.add(layers.Flatten())
glove_model.add(layers.Dense(3, activation='softmax'))
glove_model.summary()


#With the set_weights method we load the pre-trained embeddings in 
#the Embedding layer (here layer 0). By setting the trainable attribute to False, 
#we make sure not to change the pre-trained embeddings


glove_model.layers[0].set_weights([emb_matrix])
glove_model.layers[0].trainable = False


NameError: name 'GLOVE_DIM' is not defined

In [3]:
class testing():
    new_var = 5
    
    def print_n(self):
        print(self.new_var)
temp = testing()
temp.print_n()

5


In [ ]:
# CUSTOM CALLBACKS IN KERAS
# THIS CALLBACK SAVES THE MODEL NOT NEEDED BUT IDK WHY I MADE THIS
# custom callback
class save_model_1(tf.keras.callbacks.Callback):
    def __init__(self,filepath):
        self.paths = filepath
    
    min_loss = 10000000

    def on_epoch_end(self, epoch, logs=None):
        print(logs['loss'])
        print(self.paths)
        if logs["loss"] < self.min_loss:
            min_loss = logs["loss"]
            temp_model = self.model
            mem = []
            self.model.save(self.paths)
            #for i in temp_model.layers:
            #    mem.append(i.trainable)
            #    i.trainable = False
            #
            #for c,i in enumerate(temp_model):
            #    i.trainable = mem[c]
        
save_model = save_model_1("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset\Models/timepass.h5")
#print(save_model.paths)
print(type(save_model))

In [ ]:
# USING LABEL ENCODER FOR TRANSFORMATIONS

from sklearn.preprocessing import LabelEncoder
enoder_dict = {}
for i in df.columns:
    df[i]= df[i].astype('category')
    le = LabelEncoder()
    le.fit(df[i])
    enoder_dict[i] = le
    df[i] = le.transform(df[i])
    
# WILL DO THE INVERSE TRANSFORM AND BRING THE ORIGINAL WORD BACK
for c,i in enumerate(df.columns):
    x = enoder_dict[i]
    print(x.inverse_transform(df[i]))

In [ ]:
# make pie chart

temp = application_train["NAME_INCOME_TYPE"].value_counts()

plt.figure(figsize=(10,10))

plt.title("Income Type")
plt.pie(temp.values,labels = temp.index,autopct='%1.0f%%',)
plt.legend(frameon=True, bbox_to_anchor=(1.5,0.8))
plt.show()

In [ ]:
# make bar graphs
temp = application_train["OCCUPATION_TYPE"].value_counts()
plt.figure(figsize=(20,8))
plt.title("Occupation of applicants")
sns.barplot(y = temp.index,x = temp.values,orient = 'h')
plt.show()

In [ ]:
# KDE PLOT
def kde_target(var_name, df):
    
    corr = df['TARGET'].corr(df[var_name])
    
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title(f'%s Distribution {var_name}')
    plt.legend();
    
    print("The correlation is ")
    print(corr)

In [1]:
# plot colors link

https://medium.com/@morganjonesartist/color-guide-to-seaborn-palettes-da849406d44f

In [1]:
import numpy as np
import pandas as pd

In [26]:
df = pd.read_csv(r"C:\Users\TheDarkAce\Downloads\small-subset.csv",skipinitialspace=True)

In [27]:
df.head()

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,2013-01-01 15:11:48,CSH,6.5,0.0,0.5,0.0,0.0,7.0
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-06 00:18:35,CSH,6.0,0.5,0.5,0.0,0.0,7.0
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,2013-01-05 18:49:41,CSH,5.5,1.0,0.5,0.0,0.0,7.0
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,2013-01-07 23:54:15,CSH,5.0,0.5,0.5,0.0,0.0,6.0
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,2013-01-07 23:25:03,CSH,9.5,0.5,0.5,0.0,0.0,10.5


In [43]:
group = df.groupby('hack_license')

In [53]:
df.payment_type.unique()

array(['CSH', 'DIS', 'NOC', 'CRD'], dtype=object)

In [56]:
for i in df.columns:
    print(df[i].isnull().any())

False
False
False
False
False
False
False
False
False
False
False


In [57]:
df.shape

(1001, 11)